In [0]:
pip install openpyxl

In [0]:

import requests
import json
from datetime import datetime,date,timedelta
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType,IntegerType,DateType, NullType
from pyspark.sql.functions import from_json, col, schema_of_json, lit,substring,to_date, split,col,udf,transform,date_format,current_timestamp, to_timestamp, concat

In [0]:
import pandas as pd
import requests
from pathlib import Path
from datetime import datetime
import logging
import sys
import tempfile

In [0]:
dbutils.widgets.text("volume_path", "")

In [0]:
volume_path=dbutils.widgets.get("volume_path")

In [0]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


In [0]:

# Configuration
RIG_COUNT_URL = "https://rigcount.bakerhughes.com/static-files/5844d633-b1f8-45ef-bcb3-fc7a6ac3849b"
DATABASE_PATH = "rig_count_data"
BRONZE_TABLE = "bronze_rig_count_raw"


In [0]:
logger.info(f"Downloading rig count data from {RIG_COUNT_URL}")
try:
    # Download the Excel file
    response = requests.get(RIG_COUNT_URL, timeout=30)
    response.raise_for_status()
        # Save to temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx') as tmp_file:
        tmp_file.write(response.content)
        excel_file = tmp_file.name
except Exception as e:
    logger.error(f"Failed to download rig count data: {e}")
    raise

In [0]:

try:
    # Read only the NAM Weekly sheet from the Excel file
    # Manually declare column names
    column_names = [
        "Country", "County", "Basin", "GOM", "DrillFor", "Location", 
        "State/Province", "Trajectory", "Year", "Month", "US_PublishDate", "Rig Count Value"
    ]
    
    # Read starting from row 12 with manual column names
    df = pd.read_excel(
        excel_file, 
        sheet_name='NAM Weekly', 
        header=10,  # Row 11 (0-indexed as 10)
        names=column_names
    )
 
    
    # Keep bronze layer completely raw - no cleaning here
    
    logger.info(f"Read NAM Weekly data with shape: {df.shape}")
    logger.info(f"Column names: {list(df.columns)}")

    
except Exception as e:
    logger.error(f"Failed to read rig count data: {e}")
    raise

In [0]:
# parquet_output = "/Volumes/earth_data/bronze/operationaldata/rig_count"
# df.to_parquet(parquet_output, index=False)

In [0]:
# df.to_excel("/Volumes/earth_data/bronze/operationaldata/rig_count.xlsx", index=False)

In [0]:
spark_df = spark.createDataFrame(df)

spark_df.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("/Volumes/earth_data/bronze/operationaldata/rig_count")

In [0]:


# output_path = "/Volumes/earth_data/bronze/operationaldata/rig_data.xlsx"

# dbutils.fs.put(output_path, json_data, overwrite=True)